## Preliminaries

### Imports

In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.optimize
import sklearn.datasets
import sklearn.linear_model


%matplotlib inline

### Data Directories 

Create a directory with the path below

In [2]:
raw_data_dir="../../raw/california_housing"
data_dir="../../data/probabilisticTools"

### Random Seed

In [3]:
seed=2506
np.random.seed(seed)

### Get Data

<div class="alert alert-block alert-success"> Problem 0 </div>
We download the California housing dataset using the function `sklearn.datasets.fetch_california_housing`.

In [4]:
import sklearn.datasets
housing=sklearn.datasets.fetch_california_housing()

In [5]:
housing.keys()

dict_keys(['data', 'target', 'feature_names', 'DESCR'])

In [6]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block
        - HouseAge      median house age in block
        - AveRooms      average number of rooms
        - AveBedrms     average number of bedrooms
        - Population    block population
        - AveOccup      average house occupancy
        - Latitude      house block latitude
        - Longitude     house block longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
http://lib.stat.cmu.edu/datasets/

The target variable is the median house value for California districts.

This dataset was derived from the 1990 U.S. census, using one row per census
block group. A block group is the smallest geographical unit for which the U.S.
Census Bur

In [7]:
print(len(housing.feature_names),housing.feature_names)

8 ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


In [8]:
print(housing.data.shape,housing.target.shape)

(20640, 8) (20640,)


In [9]:
data=pd.DataFrame(housing.data,columns=housing.feature_names)
data["value"]=housing.target
data.describe()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,value
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


## Data Pre-Processing

The variables in `data` have very different scales.
We will replace the values  $x$ on each column by their standarized values 
$$
    z = \frac{x - \bar{x}}{\sigma_x}
$$

<div class="alert alert-block alert-info"> Problem 1.1 </div>
Compute the mean and std deviation of each column in `data`

[HINT] Pandas has convenient functions to compute the column mean an std deviation

In [10]:
data.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,value
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [11]:
mean_ = data.mean(axis=0)
mean_

MedInc           3.870671
HouseAge        28.639486
AveRooms         5.429000
AveBedrms        1.096675
Population    1425.476744
AveOccup         3.070655
Latitude        35.631861
Longitude     -119.569704
value            2.068558
dtype: float64

In [12]:
std_ = data.std(axis=0)
std_

MedInc           1.899822
HouseAge        12.585558
AveRooms         2.474173
AveBedrms        0.473911
Population    1132.462122
AveOccup        10.386050
Latitude         2.135952
Longitude        2.003532
value            1.153956
dtype: float64

<div class="alert alert-block alert-info"> Problem 1.2 </div>
Create a new `DataFrame` called `data_standarized` the value $x$ of each column gets replaced by its standarized value 
$$
    z = \frac{x - \bar{x}}{\sigma_x}
$$

In [13]:
data_standarized = (data-mean_)/std_

In [14]:
data_standarized.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,value
0,2.344709,0.982119,0.628544,-0.153754,-0.974405,-0.049595,1.052523,-1.327803,2.129580
1,2.332181,-0.607004,0.327033,-0.263329,0.861418,-0.092510,1.043159,-1.322812,1.314124
2,1.782656,1.856137,1.155592,-0.049015,-0.820757,-0.025842,1.038478,-1.332794,1.258663
3,0.932945,1.856137,0.156962,-0.049832,-0.766010,-0.050328,1.038478,-1.337785,1.165072
4,-0.012881,1.856137,0.344702,-0.032905,-0.759828,-0.085614,1.038478,-1.337785,1.172871


<div class="alert alert-block alert-info"> Problem 1.3</div>
1. Create a numpy array variable named `X` with all the features (but excluding the house values)
2. Create a numpay array variable named `Y` with the house prices (values)

In [15]:
X = data_standarized.drop(columns='value').values
print(X)
print(X.shape)

[[ 2.34470896  0.98211887  0.62854423 ... -0.04959533  1.05252278
  -1.32780305]
 [ 2.33218146 -0.60700421  0.32703343 ... -0.09250999  1.04315928
  -1.32281187]
 [ 1.78265622  1.85613656  1.15559247 ... -0.0258419   1.03847753
  -1.33279424]
 ...
 [-1.14256563 -0.92482882 -0.09031584 ... -0.07173277  1.77819439
  -0.82369324]
 [-1.05455737 -0.84537267 -0.04021014 ... -0.09122294  1.77819439
  -0.87360511]
 [-0.78011057 -1.00428498 -0.07044081 ... -0.0436811   1.75010387
  -0.83367562]]
(20640, 8)


In [16]:
Y = data_standarized['value'].values
print(Y)
print(Y.shape)

[ 2.12957989  1.3141243   1.25866292 ... -0.99272244 -1.05858282
 -1.01785337]
(20640,)


## Exact Solution with Numpy

We assume a linear model
$$
     y = \sum_d x_d \theta_d  + \epsilon
$$
where $d$ runs through the housing features and $\epsilon$ is a Gaussian noise term.

<div class="alert alert-block alert-info"> Problem 2.1 </div>
Can you find a reason why we have not included a bias term `b` in the equation?

**Answer:** Because we have standardized our data, expected mean is 0. There should not exist biased term in linear model. 

<div class="alert alert-block alert-info"> Problem 2.1 </div>
Using only `numpy` matrix algebra functions, find the Maximum Likelihood values of $\theta_d$

[Hint] Computing matrix inverses is computationally expensive.  The function `numpy.lialg.solve` can be used to solve systems of linear equations.

In [17]:
A = np.dot(np.transpose(X), X)
B = np.dot(np.transpose(X), Y)
Theta_ = np.linalg.solve(A, B)
Theta_

array([ 0.71895227,  0.10291078, -0.23010693,  0.26491789, -0.00390232,
       -0.03408034, -0.77984545, -0.75441522])

<div class="alert alert-block alert-info"> Problem 2.2 </div>
Create a variable named `Y_pred` that for each sample $X$, constains  the maximum likelihood model predicted value for $Y$

In [18]:
Y_pred = np.dot(X, Theta_)

In [19]:
Y_pred

array([ 1.78784232,  1.65348419,  1.39347822, ..., -1.64417577,
       -1.51604801, -1.34559232])

## Gradient Descent Optimization

We will now solve the same problem using Gradient Descent, instead of the analytic solution.

<div class="alert alert-block alert-info"> Problem 3.1 </div>
Define a python function `mse(theta,X,Y)` that computes the mean square error function given $\theta$, $X$ and $Y$

In [20]:
def mse(theta, X, Y):
    n = X.shape[0]
    return 0.5*sum((Y-np.dot(X, theta))**2)/n

<div class="alert alert-block alert-info"> Problem 3.2 </div>
Define a python function `grad(theta,X,Y)` that computes the gradient of the error function given $\theta$, $X$ and $Y$

In [21]:
def grad(theta,H,Y):
    Y_pred=np.dot(H,theta)
    dY=(Y_pred-Y)
    return np.dot(H.T,dY)/len(H)

<div class="alert alert-block alert-info"> Problem 3.4 </div>
Using [`numpy.random.normal`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.random.normal.html) 
generate a random guess of the vector $\theta$ so that each component is $\mathcal{N}(0,1)$ distributed

In [22]:
theta_random = np.random.normal(size=8)

In [23]:
theta_random

array([ 1.18332114,  1.3144417 , -1.43101293, -0.90304957,  1.47576176,
       -1.11638228,  1.71375566, -0.16266391])

<div class="alert alert-block alert-info"> Problem 3.3 </div>
Use the function [`scipy.optimze.check_grad`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.check_grad.html)
to verify numerically that `grad` is really the gradient of `mse` for the  $\theta$ guess.

[HINT] `grad` is the gradient of `mse` if `check_grad` returns a very small number (say $\approx 10^-8$)

In [24]:
scipy.optimize.check_grad(mse,grad,theta_random,X,Y)

3.7217404515717576e-06

<div class="alert alert-block alert-info"> Problem 3.4 </div>
** Steepest Descent Algorithm**

1. Pick a value for the learning rate $\eta$
1. Implement the steepest descent update rule
    $$
        \theta \leftarrow \theta - \eta \frac{\partial E}{\partial \theta}
    $$
1. Run the update rule on a loop, starting from your random guess for $\theta$. Repeat  $T=1000$ times
1. Every 100 steps, print the step number and the current error
1. After 1000 steps, print the final error, and the final $\theta$ parameters.
2. If process did not converge, modify value of learning rate $\eta$ and repeat until convergence.

In [25]:
n_ = 0.25
theta_ = theta_random
for i in range(1, 1001):
    theta_ = theta_ - n_*grad(theta_, X, Y)
    if i%100 == 0:
        error = mse(theta_, X, Y)
        print(f'-- Current step is {i}, Error is{error}')
        print('')
print(f'Final error is {mse(theta_, X, Y)}')
print(f'Final theta values are {theta_}')

-- Current step is 100, Error is0.20736237222949055

-- Current step is 200, Error is0.19782334474190083

-- Current step is 300, Error is0.1969682374520031

-- Current step is 400, Error is0.19688359927224905

-- Current step is 500, Error is0.19687507594024656

-- Current step is 600, Error is0.19687421519975012

-- Current step is 700, Error is0.1968741282374114

-- Current step is 800, Error is0.19687411945079544

-- Current step is 900, Error is0.1968741185629923

-- Current step is 1000, Error is0.19687411847328726

Final error is 0.19687411847328726
Final theta values are [ 0.71895737  0.10291172 -0.23011658  0.26492588 -0.00390205 -0.03408053
 -0.77983378 -0.75440415]


<div class="alert alert-block alert-info"> Problem 3.5 </div>
Compare the MSE of the steepest descent solution to the exact solution.

In [26]:
print(f'Steepest Descent MSE is {mse(theta_, X, Y)}')
print(f'Exact Solution MSE is {mse(Theta_, X, Y)}')
print('They have basically the same mse.')

Steepest Descent MSE is 0.19687411847328726
Exact Solution MSE is 0.19687411846320496
They have basically the same mse.


<div class="alert alert-block alert-info"> Problem 3.6 </div>
Compare the  steepest descent parameters $\theta$  to the exact solution.

In [27]:
print(f'Steepest Descent theta is {theta_}')
print(f'Exact theta is {Theta_}')
print('We can observe that they are exactly the same')

Steepest Descent theta is [ 0.71895737  0.10291172 -0.23011658  0.26492588 -0.00390205 -0.03408053
 -0.77983378 -0.75440415]
Exact theta is [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]
We can observe that they are exactly the same


## Sklearn Comparison

<div class="alert alert-block alert-info"> Problem 4.1 </div>
Use [`sklearn.linear_model.LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)
to fit our model.

[Hint] You will need to create a `LinearRegression` object, and the call the `fit` method. Make sure not to fit the intercept (bias).


In [28]:
lr = sklearn.linear_model.LinearRegression(fit_intercept=False)
reg = lr.fit(X, Y)

<div class="alert alert-block alert-info"> Problem 4.2 </div>
Compute the mean squared different between the exact model prediction's  `Y_pred`  we saved before and
`sklearn`'s Linear model regression predictions

In [29]:
Y_lr = reg.predict(X)
msd = np.mean((Y_lr-Y_pred)**2)
msd

8.29548393171549e-31

<div class="alert alert-block alert-info"> Problem 4.3 </div>
Compare the sklearn solution to the exact solution we found earlier.

[Hint] The solution is stored on the model's  `coef_` variable

In [30]:
print(f'Sklearn solution is {lr.coef_}')
print(f'Exact solution is {Y_pred}')
print('There are no differences.')

Sklearn solution is [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]
Exact solution is [ 1.78784232  1.65348419  1.39347822 ... -1.64417577 -1.51604801
 -1.34559232]
There are no differences.


### Statmodels  Comparison

In [31]:
import statsmodels.api as sm

We will solve using  `statmodels` so that we appreciate the difference in emphasis between Machine Learning (`sklearn`) and Statistics Modeling `statmodels` 

<div class="alert alert-block alert-info"> Problem 5.1 </div>
Use [`statmodels.api.OLS`](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.html) to solve the same linear regression problem


In [32]:
model = sm.OLS(Y,X)
result = model.fit()
Y_statpred = result.predict(X)

<div class="alert alert-block alert-info"> Problem 5.2 </div>
Compare the `statmodels` solution to the exact solution we found earlier.

[Hint] The fitted parameters are stored on the results 's  `parms` variable

In [33]:
print(f'The statmodels solution is: {result.params}')
print(f'The exact solution is {Theta_}')
print('There are no obvious differences.')

The statmodels solution is: [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]
The exact solution is [ 0.71895227  0.10291078 -0.23010693  0.26491789 -0.00390232 -0.03408034
 -0.77984545 -0.75441522]
There are no obvious differences.


<div class="alert alert-block alert-info"> Problem 5.3 </div>
Print a  `statmodels` result summary (function `summary` of the results object).

It will show you a number of estimates on goodness-of-fit, significance of coefficients, etc.

In [34]:
print(result.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.606
Model:                            OLS   Adj. R-squared (uncentered):              0.606
Method:                 Least Squares   F-statistic:                              3971.
Date:                Sun, 09 Feb 2020   Prob (F-statistic):                        0.00
Time:                        15:40:46   Log-Likelihood:                         -19668.
No. Observations:               20640   AIC:                                  3.935e+04
Df Residuals:                   20632   BIC:                                  3.942e+04
Df Model:                           8                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

### Independent test for categorial variables

<div class="alert alert-block alert-info"> Problem 6.1 </div>

Read the data from file 'homework.csv' in the  'data_dir' directory (you must run `GenerateCategorialVariables` notebook first to create the data)

Perform a $\chi^2$ test of independence between the variables `X` and `Y`.
Are 'X' and 'Y' dependent on each other?

[Hint] You can copy any code you need from the [`CategoricalInference`](./CategoricalInference.ipynb) Notebook,
but make sure to import any python modules you may need.

In [35]:
import sys 
import scipy.special as special
sys.path.append("../..")
data_dir="../../data/ProbabilisticTools"
data_gcv = pd.read_csv(data_dir +'/homework.csv')

In [36]:
data_gcv.head()

,X,Y
0,B,e
1,A,d
2,B,c
3,B,e
4,B,c


In [37]:
X_test = data_gcv['X']
Y_test = data_gcv['Y']

In [38]:
Z_x = pd.get_dummies(X_test)
Z_x = Z_x.values

Z_y = pd.get_dummies(Y_test)
Z_y = Z_y.values

In [39]:
def C2_independence(Z_x,Z_y):
    N=len(Z_x)
    D=Z_x.shape[1]
    K=Z_y.shape[1]
    # p_y has index k
    p_y=Z_y.mean(axis=0)
    # p_x has index d
    p_x=Z_x.mean(axis=0)
    # p will be K*D, with indexes k,d
    p=p_y[:,np.newaxis]*p_x[np.newaxis,:]
    # expectation if x and y are independent
    expect=N*p
    # Z_y has indexes i,k and Z_x has indexes i,d
    #Z will be N*K*D, with indexes i,k,d
    Z=Z_y[:,:,np.newaxis]*Z_x[:,np.newaxis,:]
    # observations for each (y,x) 
    # sum over i, left with a K*D matrix
    obs=Z.sum(axis=0) # last two expressions are the same as np.dot(Z_y^T,Z_x)
    df=obs-expect
    df2=df*df
    # we need to special case 0/0 case.
    c2 = (df2/np.maximum(1e-9,expect)).sum()
    return c2,special.chdtrc((K-1)*(D-1),c2)

In [40]:
C2_independence(Z_x,Z_y)

(9.637411597313756, 0.6477364094019291)

#### We can not reject the null hypothesis that Z_x and Z_y are independent. 

H_0: Z_x, Z_y are independent